# 13.6 Lab: Multiple Testing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.metrics import confusion_matrix
from statsmodels.sandbox.stats.multicomp import multipletests
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import TukeyHSDResults

import json

%matplotlib inline


## 13.6.1 Review of Hypothesis Tests

In [ ]:
np.random.seed(21)
X = np.random.normal(loc=0.0, scale=1.0, size=(10, 100))
offset = 0.5
X[:,:50] = X[:,:50] + offset

In [ ]:
# here I used scipy. During google search, I came across bioinfokit module, could explore more. 
result=st.ttest_1samp(a = X[:, 0], popmean = 0)
print(result.pvalue)

In [ ]:
# let us run the same t-test for all 100 columns
p_values = []
decision = []
for i in range(100):
    result=st.ttest_1samp(a = X[:, i], popmean = 0)
    p_values.append(result.pvalue)
    if result.pvalue < 0.05:
        decision.append('Reject H0')
    else:
        decision.append('Do not reject H0')


In [ ]:
# after computing the p-values, we can use the ground truth to evaluate the performance
ground_truth = np.repeat(['Reject H0', 'Do not reject H0'], [50, 50], axis=0)
labels = ['Reject H0', 'Do not reject H0']
cm = confusion_matrix (ground_truth, decision, labels=labels)
print(cm)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('One sample t-test')
plt.ylabel('Ground truth')
plt.show()

In [ ]:
# we could make the offset larger (from 0.5 to 1) and see the change to the confusion matrix
offset = 1
X[:,:50] = X[:,:50] + offset

p_values = []
decision = []
for i in range(100):
    result=st.ttest_1samp(a = X[:, i], popmean = 0)
    p_values.append(result.pvalue)
    if result.pvalue < 0.05:
        decision.append('Reject H0')
    else:
        decision.append('Do not reject H0')


ground_truth = np.repeat(['Reject H0', 'Do not reject H0'], [50, 50], axis=0)
labels = ['Reject H0', 'Do not reject H0']
cm = confusion_matrix (ground_truth, decision, labels=labels)
print(cm)

## 13.6.2 The Family-Wise Error Rate

In [ ]:
m = range(500)
fwe1 = list(map(lambda x:1 - pow(1 - 0.05,x),m))
fwe2 = list(map(lambda x:1 - pow(1 - 0.01,x),m))
fwe3 = list(map(lambda x:1 - pow(1 - 0.001,x),m))

In [ ]:
plt.plot(m, fwe1, label = "0.05")
plt.plot(m, fwe2, label = "0.01")
plt.plot(m, fwe3, label = "0.001")
plt.xlabel('Number of tests in log scale')
plt.ylabel('FWE')
plt.xscale("log")
plt.legend()
plt.show()

""" 
We see that setting α = 0.05 results in a high FWER even for moderate m. 
With α = 0.01, we can test no more than five null hypotheses before the FWER exceeds 0.05. 
Only for very small values, such as α = 0.001, do we manage to ensure a small FWER, 
at least for moderately-sized m.

Of course, the problem with setting α to such a low value is that we are likely to 
make a number of Type II errors: in other words, our power is very low.
"""

In [ ]:
Fund = pd.read_csv('data/Fund.csv')

In [ ]:
Fund.head()

In [ ]:
# we will do the one sample t test for the first manager
result=st.ttest_1samp(a = Fund['Manager1'], popmean = 0)
print(result.pvalue)

In [ ]:
p_values = []
manager_number = 5 

for i in range(manager_number):
    result=st.ttest_1samp(a = Fund.iloc[:,i], popmean = 0)
    p_values.append(result.pvalue)

print(p_values)

""" 
The p-values are low for Managers One and Three, and high for the other three managers. 
However, we cannot simply reject H01 and H03, since this would fail to account for 
the multiple testing that we have performed. 
Instead, we will conduct Bonferroni’s method and Holm’s method to control the FWER.
"""

In [ ]:
# we could bonferroni to adjust the raw p-values and take care of family wise error rate
reject, p_values_corrected, alphacSidak, alphacBonf = multipletests(p_values, method = 'bonferroni')
print(p_values_corrected)
""" 
Therefore, using Bonferroni’s method, 
we are able to reject the null hypothesis only for Manager One while controlling the FWER at 0.05.
This information is also available in the variable reject.
"""
print(reject)


In [ ]:
# Bonferroni's method is more conservative. We could apply holm's method to control the FWER
reject, p_values_corrected, alphacSidak, alphacBonf = multipletests(p_values, method = 'holm')
print(p_values_corrected)
print(reject)
""" 
By contrast, using Holm’s method, the adjusted p-values indicate that we can reject the null hypotheses 
for Both Managers One and Three at a FWER of 0.05.
"""

In [ ]:
# we can see the average for each manager 
Fund.mean(axis=0)

In [ ]:
# next, we could test whether 2 managers are significantly different. For example Manager 1 and Manager 2
result=st.ttest_rel(a = Fund['Manager1'], b = Fund['Manager2'])
print(result.pvalue)

In [ ]:
""" 
However, we decided to perform this test only after examining the data and 
noting that Managers One and Two had the highest and lowest mean performances. 
In a sense, this means that we have implicitly performed a manual selection 
from the 5(5 − 1)/2 = 10 hypothesis tests, rather than just one. 
Hence, we use Tukey’s method in order to adjust for multiple testing. 
"""
returns = Fund.iloc[:, :5].to_numpy().flatten(order='F') # we flatten by col (i.e. order='F')
manager = np.repeat(['1', '2', '3', '4', '5'], repeats=Fund.shape[0])

# perform Tukey's test
tukey = pairwise_tukeyhsd(endog=returns, groups=manager, alpha=0.05)

print(tukey)

""" 
Notice that the p-value for the difference between Managers One and Two has increased from 0.038 to 0.186, 
so there is no longer clear evidence of a difference between the managers’ performances.

"""

## 13.6.3 The False Discovery Rate

In [ ]:
p_values = []
manager_number = Fund.shape[1]

for i in range(manager_number):
    result=st.ttest_1samp(a = Fund.iloc[:,i], popmean = 0)
    p_values.append(result.pvalue)

print(p_values[0:10])

In [ ]:
""" 
There are far too many managers to consider trying to control the FWER. 
Instead, we focus on controlling the FDR: that is, the expected fraction of rejected null 
hypotheses that are actually false positives. 
"""

reject, p_values_corrected, alphacSidak, alphacBonf = multipletests(p_values, method = 'fdr_bh')
print(p_values_corrected[0:10])

""" 
The q-values output by the Benjamini-Hochberg procedure can be interpreted as the smallest 
FDR threshold at which we would reject a particular null hypothesis.

For instance, a q-value of 0.1 indicates that we can reject the corresponding null hypothesis
at an FDR of 10% or greater, but that we cannot reject the null hypothesis at an FDR below 10%.
"""

In [ ]:
# we would find that 146 of the 2,000 fund managers have a p_values_corrected below 0.1
sum(p_values_corrected <= .1)

In [ ]:
# if we use bonferroni method, we will find None
sum(np.array(p_values) <= .1/Fund.shape[1])

## 13.6.4 A Re-Sampling Approach

In [ ]:
# I saved the gene expression data as a json file, in python we could load the json file using the json library
# after reading in the data, we can use the data is same as a dictionary, we can use the keys to access the data

f = open('./data/Khan.json')
Khan = json.load(f)

X_train = np.array(Khan['xtrain'])
y_train = np.array(Khan['ytrain'])
X_test = np.array(Khan['xtest'])
y_test = np.array(Khan['ytest'])

In [ ]:
x = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)
unique, counts = np.unique(y, return_counts=True)
print(counts)

In [ ]:
# x1: take the x for cancer type == 2
# x2: take the x for cancer type == 4
x1 = x[y==2, :]
x2 = x[y==4, :]
n1 = x1.shape[0]
n2 = x2.shape[0]
print(n1)
print(n2)

In [ ]:
# performing a standard two-sample t-test on the 11th (gene_index = 10 in python) gene produces a test-statistic 
gene_index = 10
original_result=st.ttest_ind(a=x1[:,gene_index], b=x2[:,gene_index], equal_var=True)
print(original_result.statistic)
print(original_result.pvalue)

""" 
The 2 sample t-test produces a test-statistic of −2.09 and an associated p-value of 0.0412, 
suggesting modest evidence of a difference in mean expression levels between the two cancer types.
"""

In [ ]:
""" 
Instead of doing a parameterized 2 sample t-test, we could do a non-parameterized test(i.e. permutation test).
we can randomly split the 54 patients (in cancer group 2 and 4) into two groups of 29 and 25 
(same as the original split),and compute a new test statistic. 
Under the null hypothesis of no difference between the groups, this new test statistic should have 
the same distribution as our original one. 
Repeating this process many (i.e.10,000) times allows us to approximate the null distribution of the test statistic. 
We compute the fraction of the time that our observed test statistic exceeds the test statistics obtained 
via re-sampling.
"""

np.random.seed(21)
iteration = 10000
test_stats = []
x_temp = np.concatenate((x1[:,gene_index], x2[:,gene_index]), axis=0)

for i in range(iteration):
    np.random.shuffle(x_temp)
    result_temp = st.ttest_ind(a=x_temp[:n1], b=x_temp[-n2:], equal_var=True)
    test_stats.append(result_temp.statistic)

In [ ]:
print(np.mean((np.abs(test_stats) >= np.abs(original_result.statistic))))

""" 
This fraction is our re-sampling-based p-value. It is almost identical to the p-value of 0.0412 
obtained using the theoretical null distribution.

The reason for this is that the parametrized distribution is a pretty good assumption in this case
To see this, we can plot the histogram of the re-sampled statistics vs. parametrized distribution. 

We could try other genes (i.e. gene_index = 876) to see its theoretical and re-sampling null distributions are 
quite different
"""

In [ ]:
# construct the t distribution 
df = n1 + n2 - 2
rv = st.t(df)
x = np.linspace(-4.2, 4.2, 1000)


plt.hist(test_stats, 100, density=True, facecolor='g', alpha=0.75)
plt.plot(x, rv.pdf(x), 'k-', lw=2)
plt.xlabel('Null Distribution of Test Statistic')
plt.ylabel('Probability')
plt.title('Histogram of re-sample stats')
plt.xlim(-4.2, 4.2)
plt.grid(True)
plt.show()

In [ ]:
# we could do this for 100 and see how FDR works under re-sample 
# it would be good to use small iterations to make sure the code runs okay 
num_gene = 100
iteration = 500
test_stats_matrix = []
test_stats_origin = []

for j in range(num_gene):
    gene_index = j 
    x_temp = np.concatenate((x1[:,gene_index], x2[:,gene_index]), axis=0)
    result_origin = st.ttest_ind(a=x1[:,gene_index], b=x2[:,gene_index], equal_var=True)
    test_stats_origin.append(result_origin.statistic)
    test_stats = []
    for i in range(iteration):
        np.random.shuffle(x_temp)
        result_temp = st.ttest_ind(a=x_temp[:n1], b=x_temp[-n2:], equal_var=True)
        test_stats.append(result_temp.statistic)
        
    test_stats_matrix.append(test_stats)

In [ ]:
test_stats_origin_sorted =  np.sort(np.abs(test_stats_origin))

In [ ]:
Rs = []
Vs = []
FDRs = []
for j in range(num_gene):
    R = np.sum(np.abs(test_stats_origin) >= test_stats_origin_sorted[j])
    V = np.sum(np.abs(test_stats_matrix) >= test_stats_origin_sorted[j]) / iteration
    Rs.append(R)
    Vs.append(V)
    FDRs.append(V*1.0/R)

Rs = np.array(Rs)
Vs = np.array(Vs)
FDRs = np.array(FDRs) 

In [ ]:
print(np.max(Rs[FDRs <= .1]))
print(np.max(Rs[FDRs <= .2]))

In [ ]:
plt.plot(Rs, FDRs, 'k-', lw=2)
plt.xlabel('Number of Rejections')
plt.ylabel('False Discovery Rate')
plt.show()

In [ ]:
# End of Chapter 13